In [ ]:
import csv
import os
import pandas as pd
import re
import time
from datetime import datetime
from pathlib import *
import random
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from pyvirtualdisplay import Display
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, \
    WebDriverException, StaleElementReferenceException, ElementNotInteractableException,\
    InvalidArgumentException, UnexpectedAlertPresentException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import socket
import time
from ping3 import ping, verbose_ping
import pytz


# Start a virtual display
display = Display(visible=0, size=(1920, 1080))
display.start()


# Define the Vietnam timezone
local_tz = pytz.timezone('Asia/Ho_Chi_Minh')
# Get the current time in UTC
now_utc = datetime.now(pytz.utc)
# Convert UTC time to local time
now = now_utc.astimezone(local_tz).date().strftime('%Y-%m-%d')
now_1 = now_utc.astimezone(local_tz).strftime('%Y-%m-%d')


class LABOR:
    def __init__(self, list_link: list, start_link: str):
        self.list_link = list_link
        self.start_link = start_link
        self.list_province_temp = None
        self.driver = self.chrome_drive(chrome_path)
        self.wait = WebDriverWait(self.driver, 50)
        self.action = ActionChains(self.driver)
        self.Brand = "Apple"
        self.file_name = f"../a_output/tz-apple_{now}.csv"
        for link in self.list_link[list_link.index(start_link):]:
            print(link)
            self.list_province_temp = None  # At the beginning of every link, set it none
            self.restart_province = None  # At the beginning of every link, set it none
            self.check(link, list_province_to_check=list_province)
        self.driver.quit()


    def chrome_drive():
    
        # Configure Chrome options
        options = webdriver.ChromeOptions()
        # options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
        options.add_argument('--no-sandbox')  # Bypass OS security model
        options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    
        # Create a driver instance
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    
        # Disable the "Chrome is being controlled by automated test software" notification
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
        # # Disable the "navigator.webdriver" property
        options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    
        # Disable the "Chrome is being controlled by automated test software" banner
        options.add_argument("--disable-blink-features=AutomationControlled")
    
        # Maximize the window to avoid fingerprinting based on screen resolution
        options.add_argument("start-maximized")
    
        # Disabling the Automation Extension can help prevent detection as an automated script and increase the chances of
        # successfully completing your automation tasks.
        options.add_experimental_option('useAutomationExtension', False)
    
        # This argument tells the browser to ignore any SSL certificate errors that may occur while accessing a website,
        # which is useful when testing on a site with a self-signed or invalid SSL certificate. Without this argument,
        # the browser will display a warning message about the certificate and require manual confirmation to proceed.
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
    
        # wait for the page to be fully loaded before proceeding
        # options.page_load_strategy = 'normal'  # 'none', 'eager', or 'normal'
    
        prefs={
            "disable-transitions": True,
            "profile.managed_default_content_settings.images": 2,
            "profile.default_content_setting_values.notifications": 2
        }
        options.add_experimental_option("prefs", prefs)
    
        return driver
    
    
    def check_internet_connection(self):
        host = "8.8.8.8"  # Google's public DNS server
        max_attempts = 5
        interval = 10  # in seconds

        for _ in range(max_attempts):
            response_time = ping(host)
            if response_time is not None:
                # print("Internet connection is active.")
                return True
            else:
                print("No internet connection. Retrying in {} seconds...".format(interval))
                time.sleep(interval)

        print("No internet connection after multiple attempts.")
        return False

    def click_seemore(self):
        while True:
            try:
                self.wait.until(
                    EC.presence_of_element_located((By.XPATH, "//ul[@id='listAvailableStores']/following-sibling::a")))
                seemore_but = self.driver.find_element(By.XPATH, "//ul[@id='listAvailableStores']/following-sibling::a")
                time.sleep(1)
                self.driver.execute_script("arguments[0].scrollIntoView(true);", seemore_but)

                self.action.move_to_element(seemore_but).click().perform()
                time.sleep(1)

            except (ElementNotInteractableException, NoSuchElementException, TimeoutException):
                break
            except StaleElementReferenceException:
                continue

    def remove_unwanted_product(self):
        try:
            cart_clear_elements = self.driver.find_elements(By.XPATH,
                                                            "//ul[@class='listing-cart']/li/div[1]/div/button/span")

            if len(cart_clear_elements) > 1:
                while len(cart_clear_elements) > 1:
                    cart_clear = cart_clear_elements[-1]  # Click the last element
                    try:
                        # Scroll to the element and click it
                        self.driver.execute_script("arguments[0].scrollIntoView(true);", cart_clear)
                        self.driver.execute_script("arguments[0].click();", cart_clear)
                        print("Removed a product.")

                        # Wait briefly for the page to update after removal
                        time.sleep(2)

                        # Refresh the list of elements after removal
                        cart_clear_elements = self.driver.find_elements(By.XPATH,
                                                                        "//div[@class='product-cart__remove']")
                    except (ElementNotInteractableException, StaleElementReferenceException):
                        # If the element cannot be interacted with or becomes stale, move to the next one
                        pass
            else:
                print("Only one product remaining. No removal needed.")

        except NoSuchElementException:
            print("No product removal needed.")

    def delete_old(self):
        while True:
            try:
                cart_clear = self.driver.find_element(By.XPATH, "//ul[@class='listing-cart']/li/div[1]/div/button/span")
                time.sleep(1)
                self.driver.execute_script("arguments[0].scrollIntoView(true);", cart_clear)
                time.sleep(.5)

                # Execute JavaScript to click the element
                self.driver.execute_script("arguments[0].click();", cart_clear)

            except (
            ElementNotInteractableException, NoSuchElementException, TimeoutException, InvalidArgumentException):
                break  # Break the loop if the element cannot be interacted with or is not found
            except StaleElementReferenceException:
                continue  # Continue the loop to make another attempt

    def record(self, data_list):
        if not os.path.exists("../a_output"):
            os.makedirs(f"../a_output")

        with open(self.file_name, "a") as file:
            writer = csv.DictWriter(file,
                                    fieldnames=["ProductName", "Brand", "Channel", "Province", "StoreName",
                                                "GoogleLink", "Color", "Stock", "Promotion_Price",
                                                "Additional_Promotion", "Note", "Special_Note", "Link", "Date"],
                                    delimiter=";")
            if os.stat(self.file_name).st_size == 0:
                writer.writeheader()
            for row in data_list:
                writer.writerow(row)

    def no_internet_reload(self, link, list_province_to_check):
        # Call the function to check for internet connection before proceeding
        if self.check_internet_connection():
            pass
        else:
            print("no_internet_reload(restart_link)")
            self.check(link, list_province_to_check)

    def check(self, link, list_province_to_check):
        try:
            # It must be clean before getting the link
            self.driver.delete_all_cookies()
            print("self.restart_province: ", self.restart_province)

            self.driver.get(link)

            time.sleep(7)

            try:
                self.driver.find_element(By.TAG_NAME, "h1")
            except NoSuchElementException:
                ProductName = f"Please double check the link: {link}"
                print(ProductName)
                ton_kho = "No"
            else:
                # If the product has the class="capacity", use this capacity to add to product name
                try:
                    self.driver.find_element(By.CLASS_NAME, "capacity")

                except NoSuchElementException:
                    ProductName = link[link.rindex("/") + 1:].replace("-", " ").replace("iphone", "iPhone").replace("pro",
                                                                                                                     "Pro").replace(
                        "gb", "GB").replace("tb", "TB").replace("max", "Max").replace("macbook", "MacBook").replace("ipad",
                                                                                                                    "iPad").replace(
                        "apple", "Apple").replace("watch", "Watch")
                else:
                    capacities = self.driver.find_elements(By.CSS_SELECTOR, ".capacity ul > li")
                    capacity_text = str
                    for capacity in capacities:
                        if capacity.get_attribute("class") == "active":
                            capacity_text = capacity.text
                        else:
                            pass
                    try:
                        ProductName = self.driver.find_element(By.TAG_NAME, "h1").text.replace(" Mới", "") + " " + capacity_text
                    except TypeError:
                        ProductName = link[link.rindex("/") + 1:]
                    print(ProductName)

                try:
                    self.driver.find_element(By.CLASS_NAME, "btn-buy")
                except NoSuchElementException:
                    ton_kho = "No"
                else:
                    ton_kho = "Yes"
                print(ton_kho)

            if ton_kho == "Yes":
                def click_buy_but():
                    buy_but=self.driver.find_element(By.XPATH, "(//a[contains(@class, 'btn-buy')])[last()]")
                    self.driver.execute_script("arguments[0].scrollIntoView(true);", buy_but)
                    self.driver.execute_script("arguments[0].click();", buy_but)

                def method_1():

                    # Try if the close notification appear and invisible
                    try:
                        WebDriverWait(self.driver, 15).until(
                            EC.invisibility_of_element((By.XPATH, "(//div[@class='alert-promo']/div)[1]")))
                    except TimeoutException:
                        print("(//div[@class='alert-promo']/div)[1]")
                        close_noti = self.driver.find_element(By.XPATH, "(//div[@class='alert-promo']/div)[1]")
                        self.driver.execute_script("arguments[0].scrollIntoView(true);",
                                                   close_noti)
                        self.driver.execute_script("arguments[0].click();", close_noti)
                        time.sleep(1)
                        click_buy_but()
                    else:
                        pass
                    print("Go with the method 1")

                    # Choose go to physical store
                    print("# Choose go to physical store")
                    self.wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='click-choose']/div[2]/i")))
                    chose_onsite = self.driver.find_element(By.XPATH, "//div[@class='click-choose']/div[2]/i")
                    try:
                        print("try physical store")
                        # self.action.move_to_element(chose_onsite).click().perform()
                        self.driver.execute_script("arguments[0].scrollIntoView(true);", chose_onsite)
                        self.driver.execute_script("arguments[0].click();", chose_onsite)
                        time.sleep(2)


                    except ElementNotInteractableException:
                        print("try physical store ElementNotInteractableException")

                    outer_loop_break = False  # Flag to control the outer loop
                    # click to the province name
                    for province in list_province_to_check:
                        # Handle internet interupted
                        self.restart_province = province
                        self.list_province_temp = list_province[list_province.index(self.restart_province):]
                        self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                        # choose province
                        max_retry = 3  # Maximum number of retries
                        retry_count = 0  # Counter for retries

                        while retry_count < max_retry:
                            # Click on the province_field to show provinces
                            province_field = self.driver.find_element(By.XPATH,
                                                                      "(//div[@class='btn-click country'])[2]/button")
                            try:

                                # self.action.move_to_element(province_field).click().perform()
                                self.driver.execute_script("arguments[0].scrollIntoView(true);", province_field)
                                self.driver.execute_script("arguments[0].click();", province_field)
                                time.sleep(2)  # Wait after clicking
                                self.driver.find_element(By.XPATH, "//div[@class='select active']/div/input")
                                break  # If the element province_field_2 is found, exit the loop
                            except Exception as e:
                                # Handle any unexpected exceptions within this block
                                print("An unexpected error occurred:", e)
                                # If the element is not found, increment the retry_count and try again
                                retry_count += 1
                                time.sleep(2)  # Wait before the next attempt

                        # Check if the element was found or if the max retries were reached
                        if retry_count == max_retry:
                            print("Timed out after maximum retries at province_field_2.")
                            print("restart province:", province)
                            self.restart_province = province
                            self.list_province_temp = list_province[
                                                      list_province.index(self.restart_province):]
                            self.check(link, list_province_to_check=self.list_province_temp)
                            break
                        else:
                            # Continue with your code using province_field_2
                            self.wait.until(
                                EC.presence_of_element_located((By.XPATH, "//div[@class='select active']/div/input")))

                            province_field_2 = self.driver.find_element(By.XPATH,
                                                                        "//div[@class='select active']/div/input")
                            self.action.move_to_element(province_field_2).click().perform()
                            time.sleep(.7)
                            self.action.key_down(Keys.COMMAND).send_keys('a').key_up(Keys.COMMAND).send_keys(
                                Keys.DELETE).perform()
                            time.sleep(.5)

                            self.action.send_keys(province).perform()
                            # script="arguments[0].value = arguments[1];"
                            # element=self.driver.find_element(By.XPATH,
                            #                                  "//div[@class='select active']/div/input")
                            # self.driver.execute_script(script, element, province)

                            time.sleep(.8)
                            print(province)

                        try:
                            # The matched province will be shown here
                            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(
                                (By.XPATH, "//div[@class='select active']/div[2]/aside[1]/span")))
                            province_chosen = self.driver.find_element(By.XPATH,
                                                                       "//div[@class='select active']/div[2]/aside[1]/span")
                            # self.action.move_to_element(province_chosen).click().perform()
                            # time.sleep(3)
                            self.driver.execute_script("arguments[0].scrollIntoView(true);", province_chosen)
                            time.sleep(.5)
                            self.driver.execute_script("arguments[0].click();", province_chosen)
                            time.sleep(3)

                            # July 30th, 2024, some province no longer has Topzone
                            try:
                                # Wait for the element to be present in the DOM
                                no_store=WebDriverWait(self.driver, 10).until(
                                    EC.presence_of_element_located(
                                        (By.XPATH, "//div[contains(@class, 'alert-promo')]/div/following-sibling::a"))
                                )

                                # Check if the element is displayed (visible on the screen)
                                if no_store.is_displayed():
                                    # Scroll into view
                                    self.driver.execute_script("arguments[0].scrollIntoView(true);", no_store)
                                    time.sleep(0.5)

                                    # Click the element using JavaScript
                                    self.driver.execute_script("arguments[0].click();", no_store)
                                    time.sleep(2)
                                    continue
                                else:
                                    print("The element is present in the DOM but not visible on the screen.")

                            except NoSuchElementException:
                                pass

                        except TimeoutException:
                            # Some province does not show, then we pass
                            continue

                        # Choose color later
                        self.wait.until(EC.presence_of_element_located(
                            (By.XPATH, "//div[@class='sg-color']/div")))
                        # color_options = self.driver.find_elements(By.XPATH,
                        #                                          "//div[@class='product-size-and-color-selection']/aside/div/div")

                        # Extract the raw HTML content from the page
                        page_source = self.driver.page_source

                        # Create a Beautiful Soup object for parsing
                        soup = BeautifulSoup(page_source, "html.parser")

                        # Find all div elements with class 'sg-color' and nested div elements
                        color_options = soup.select("div.sg-color div")
                        color_options = [color.text for color in color_options]

                        for color in color_options:
                            print('color: ', color)
                            # Handle no internet
                            self.restart_province = province
                            self.list_province_temp = list_province[
                                                      list_province.index(self.restart_province):]
                            self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                            # If only 1 color option, not click any thing
                            if len(color_options) > 1:
                                def click_color_but():
                                    # Handle no internet
                                    self.restart_province = province
                                    self.list_province_temp = list_province[
                                                              list_province.index(self.restart_province):]
                                    self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                                    # First step, click into color option button
                                    color_but = self.driver.find_element(By.XPATH,
                                                                         "//span[@class='color-text']")
                                    self.driver.execute_script("arguments[0].scrollIntoView(true);", color_but)
                                    # note: StaleElementReferenceException
                                    self.driver.execute_script("arguments[0].click();", color_but)
                                    # self.action.move_to_element(color_but).click().perform()
                                    time.sleep(1)

                                def click_color_option():
                                    # Handle no internet
                                    self.restart_province = province
                                    self.list_province_temp = list_province[
                                                              list_province.index(self.restart_province):]
                                    self.no_internet_reload(link, list_province_to_check=self.list_province_temp)

                                    for num in range(len(color_options)):
                                        print("num: ", num)
                                        color_chosen = self.driver.find_element(By.XPATH,
                                                                                f"(//div[@class='sg-color active']/div)[{num + 1}]")

                                        if color.strip() == color_chosen.get_attribute("textContent").strip():
                                            print("click: ", color_chosen.get_attribute("textContent"))
                                            self.driver.execute_script("arguments[0].scrollIntoView(true);",
                                                                       color_chosen)
                                            time.sleep(.5)
                                            self.driver.execute_script("arguments[0].click();", color_chosen)
                                            time.sleep(1)

                                            def handle_popup():
                                                timeout = 0.5  # Shorten the initial timeout
                                                retries = 4  # Number of retries
                                                for i in range(retries):
                                                    try:
                                                        # Wait for the popup's close button to be present
                                                        close_button=WebDriverWait(self.driver, timeout).until(
                                                            EC.presence_of_element_located(
                                                                (By.XPATH,
                                                                 "//div[@class='popup-promo active']/div[@class='alert-promo']/div[1]/following-sibling::a"))
                                                        )
                                                        # Click the close button
                                                        close_button.click()
                                                        print("Popup closed successfully.")
                                                        return  # Exit the function if popup is closed
                                                    except TimeoutException:
                                                        print(
                                                            f"Popup did not appear within the given time (attempt {i + 1}/{retries}).")
                                                        time.sleep(1)  # Short sleep before retrying

                                            handle_popup()

                                            break


                                # choose color
                                max_retry_2 = 3  # Maximum number of retries
                                retry_count_2 = 0  # Counter for retries

                                while retry_count_2 < max_retry_2:
                                    try:
                                        # click on the color button to drop the color menu
                                        click_color_but()
                                        # Second step, click into color option
                                        click_color_option()
                                        break  # If the element province_field_2 is found, exit the loop
                                    except Exception as e:
                                        print("An unexpected error occurred:", e)
                                        retry_count_2 += 1
                                        print("retry_count_2 two: ", retry_count_2)
                                        time.sleep(2)

                                # Check if the element was found or if the max retries were reached
                                if retry_count_2 == max_retry_2:
                                    print("Timed out after maximum retries at color_option_chosen.")
                                    print("restart province:", province)
                                    self.restart_province = province
                                    self.list_province_temp = list_province[
                                                              list_province.index(self.restart_province):]
                                    self.check(link, list_province_to_check=self.list_province_temp)
                                    outer_loop_break = True  # Set the flag to True
                                    break  # This will break the inner loop
                                else:
                                    pass

                            try:
                                # Wait until the loading disappeared
                                self.wait.until(EC.invisibility_of_element((By.XPATH, "//div[@class='loading-cart active']")))
                            except TimeoutException:
                                self.restart_province=province
                                self.list_province_temp=list_province[
                                                        list_province.index(self.restart_province):]
                                self.check(link, list_province_to_check = self.list_province_temp)

                            # In case, the page lag and there is no store show off
                            try:
                                # Wait until the store list appears
                                self.wait.until(EC.presence_of_element_located(
                                    (By.XPATH, "//ul[@id='listAvailableStores']/li")))
                            except (NoSuchElementException, TimeoutException):
                                self.restart_province = province
                                self.list_province_temp = list_province[
                                                          list_province.index(self.restart_province):]
                                self.check(link, list_province_to_check=self.list_province_temp)

                            # Extract the raw HTML content from the page
                            page_source = self.driver.page_source

                            # Create a Beautiful Soup object for parsing
                            soup = BeautifulSoup(page_source, "html.parser")

                            # Find the list of full stores using Beautiful Soup
                            stores = soup.select("ul#listAvailableStores li")

                            # promotion price, find the element with the "data-tmp-price-discount" attribute
                            promotion_price = soup.find('div', {'data-tmp-price-discount': True})
                            original_price = soup.find('div', {'data-tmp-price-original': True})
                            if promotion_price:
                                if int(promotion_price['data-tmp-price-discount']) > 0:
                                    promotion_price = promotion_price['data-tmp-price-discount']
                                else:
                                    promotion_price = original_price['data-tmp-price-original']
                            else:
                                promotion_price = "None"

                            try:
                                additional_promotion = soup.select_one("div.promo").get_text()
                            except TypeError:
                                additional_promotion = "None"

                            data_list = []
                            for store in stores:
                                # Province = province
                                Channel = "TZ"
                                # Find the color using Beautiful Soup
                                Color = soup.select_one("div.select-color.active small").get_text(strip=True)
                                Note = store.select_one("span small").get_text(strip=True)

                                # StoreName = store.find_element(By.XPATH, "./span").text.replace(Note, "").strip()
                                StoreName = store.select_one("span").get_text(strip=True).replace(Note, "")

                                # stock_status = store.find_element(By.XPATH, "./span/small").get_attribute('class')
                                stock_status = store.select_one("span small")["class"]
                                if 'cohang' in stock_status:
                                    Stock = "Yes"
                                else:
                                    Stock = "No"
                                GoogleLink = ""
                                data = {"ProductName": ProductName, "Brand": self.Brand, "Channel": Channel,
                                        "Province": province,
                                        "StoreName": StoreName,
                                        "GoogleLink": GoogleLink, "Color": Color, "Stock": Stock, "Promotion_Price": promotion_price,
                                                "Additional_Promotion": additional_promotion, "Note": Note, "Special_Note": "None",
                                        "Link": link, "Date": now_1}
                                data_list.append(data)
                            self.record(data_list)

                        if outer_loop_break:  # Check the flag
                            print("break the province loop to avoid redundancy")
                            break  # This will break the outermost loop

                try:
                    click_buy_but()
                    time.sleep(1)
                    method_1()

                except UnexpectedAlertPresentException:

                    print("try inside the new code")
                    WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(
                        (By.XPATH, "//div[@class='district place dl-border']/strong")))
                    district_but = self.driver.find_element(
                        By.XPATH, "//div[@class='district place dl-border']/strong")
                    self.driver.execute_script("arguments[0].click();", district_but)
                    self.wait.until(
                        EC.element_to_be_clickable((By.XPATH, "(//div[@class='scroll'])[2]/aside/a")))
                    first_district = self.driver.find_element(By.XPATH, "(//div[@class='scroll'])[2]/aside/a")
                    # self.action.move_to_element(first_district).click().perform()
                    self.driver.execute_script("arguments[0].click();", first_district)
                    self.wait.until(
                        EC.element_to_be_clickable((By.XPATH, "(//div[@class='scroll'])[3]/aside/a")))
                    first_ward = self.driver.find_element(By.XPATH, "(//div[@class='scroll'])[3]/aside/a")
                    # self.action.move_to_element(first_ward).click().perform()
                    self.driver.execute_script("arguments[0].click();", first_ward)
                    time.sleep(7)

                    print("Click the buy button 1 more time")
                    click_buy_but()

                    # Continue with method_1
                    print("# Continue with method_1")
                    method_1()
            else:
                data = {"ProductName": ProductName, "Brand": self.Brand, "Channel": "TZ",
                        "Province": "not trading",
                        "StoreName": "not trading",
                        "GoogleLink": "not trading", "Color": "not trading", "Stock": 0, "Note": "not trading",
                        "Special_Note": "None", "Link": link, "Date": now_1}
                data_list = [data]
                self.record(data_list)

        except ConnectionResetError:

            print("Restart from the end of the code: ConnectionResetError")

            if self.restart_province == None:

                self.list_province_temp = list_province

            else:

                self.list_province_temp = list_province[list_province.index(self.restart_province):]

            self.check(link, list_province_to_check=self.list_province_temp)

        except WebDriverException as e:

            if "ERR_CONNECTION_TIMED_OUT" in str(e):

                print("Restart from the end of the code: ERR_CONNECTION_TIMED_OUT")

                if self.restart_province == None:

                    self.list_province_temp = list_province

                else:

                    self.list_province_temp = list_province[list_province.index(self.restart_province):]

                self.check(link, list_province_to_check=self.list_province_temp)
            # Check if the exception message contains 'Status code was: -9'
            elif 'Status code was: -9' in str(e):
                print("ChromeDriver exited unexpectedly with status code -9.")
                # Take appropriate action (e.g., retry, log the error, notify the user)
                if self.restart_province == None:

                    self.list_province_temp = list_province

                else:

                    self.list_province_temp = list_province[list_province.index(self.restart_province):]

                self.check(link, list_province_to_check=self.list_province_temp)

